In [ ]:
 import tensorflow as tf
   from tensorflow.keras import layers, models, callbacks

   # Match the image size and batch size used in preprocessing
   IMG_HEIGHT = 128
   IMG_WIDTH  = 128
   BATCH_SIZE = 32

In [ ]:
# define architecture
def build_cnn_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)):
    model = models.Sequential([
        # Block 1
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        # Block 2
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        # Block 3
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        # Flatten & Dense layers
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')  # Binary output: 0 = bike, 1 = car
    ])
    return model

# Instantiate the model
model = build_cnn_model()

# Print a summary to verify layer shapes and parameter counts
model.summary()


In [ ]:
# compile model
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss      = 'binary_crossentropy',
    metrics   = ['accuracy']
)



In [ ]:
# Early stopping: stop if val_accuracy doesn’t improve for 5 epochs
early_stop = callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

# Save the best model weights to a file
checkpoint = callbacks.ModelCheckpoint(
    'best_bike_car_model.h5',
    monitor='val_accuracy',
    save_best_only=True
)


In [ ]:
EPOCHS = 20

history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs          = EPOCHS,
    callbacks       = [early_stop, checkpoint],
    verbose         = 2
)


In [ ]:
# Load the saved weights
model.load_weights('best_bike_car_model.h5')

# Evaluate on val_ds
val_loss, val_acc = model.evaluate(val_ds, verbose=2)
print(f"Validation Accuracy: {val_acc:.4f}")


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Gather true labels and predicted labels
y_true = []
y_pred = []

for images, labels in val_ds:
    preds = model.predict(images).flatten()
    labels = labels.numpy().astype(int)
    preds_binary = (preds >= 0.5).astype(int)

    y_true.extend(labels)
    y_pred.extend(preds_binary)

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

# Classification report (precision, recall, F1)
report = classification_report(y_true, y_pred, target_names=['Bike', 'Car'])
print("Classification Report:\n", report)
